# Chat by using ollama api

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display, Markdown

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# create a system message list
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Describe a full cycle of the Recruitment process"},
]

# create a request payload
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Using Python Ollama package

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

# Summarize the web page


In [6]:
# A class to represent a Webpage Content
import requests
from bs4 import BeautifulSoup

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [17]:
webcontent = Website("https://ollama.com/blog/functions-as-tools")
print(webcontent.title)
# print(webcontent.text)

Ollama Python library 0.4 with function calling improvements · Ollama Blog


In [11]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}. \
    The contents of this website is as follows; \
    Please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    
    user_prompt += website.text
    return user_prompt

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]


In [16]:
import ollama

MODEL = "llama3.2"
ed = Website("https://ollama.com/blog/functions-as-tools")

response = ollama.chat(model=MODEL, messages=messages_for(ed))
print(response['message']['content'])

# Summary of Ollama Python Library 0.4 Release
### Function Calling Improvements and Updates

The latest release of the Ollama Python library, version 0.4, brings several improvements and updates to its function calling capabilities.

#### New Functionality

*   Functions can now be provided as tools, allowing for more flexibility in how users interact with the library.
*   Full typing support has been added throughout the library, enabling direct object access while maintaining existing functionality.
*   Examples have been updated on the Ollama Python GitHub repository to showcase the new features.

#### Key Benefits

*   Improved usability and flexibility through function calling
*   Enhanced typing support for more robust data handling
*   Updated examples demonstrate the capabilities of the library

#### News/Announcements

*   The release marks an exciting milestone in the development of the Ollama Python library, with several notable improvements and additions.

### Getting Star